## Constructing the COVID dashboard elements

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import plotly.express as px
import dash
from datetime import timedelta

### Pull latest data from VDH

In [0]:
endpoint = 'https://data.virginia.gov/resource/bre9-aqqr.json'
mypars = {'$limit': 150000}
headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:99.0) Gecko/20100101 Firefox/99.0'}
r = requests.get(endpoint, params=mypars, headers=headers)
r

In [0]:
cases = pd.json_normalize(json.loads(r.text))
cases

### Overall data cleaning

In [0]:
cases['report_date'] = pd.to_datetime(cases['report_date'])
cases['total_cases'] = cases['total_cases'].astype('int')
cases['hospitalizations'] = cases['hospitalizations'].astype('int')
cases['deaths'] = cases['deaths'].astype('int')
cases_today = cases.loc[cases['report_date'] == max(cases['report_date'])]

### Creation of per-capita dataset

In [0]:
url = "https://demographics.coopercenter.org/sites/demographics/files/media/files/2020-07/Census_2019_RaceEstimates_forVA_0.xls"
pop = pd.read_excel(url, skiprows=4)
pop = pop.loc[~pop['FIPS'].isna()]
pop['FIPS'] = pop['FIPS'] + 51000
pop['FIPS'] = pop['FIPS'].astype('int').astype('str')
pop = pop[['FIPS', 'Total Population']]
pop

In [0]:
cases_pop = pd.merge(cases_today, pop, 
                    left_on = ['fips'],
                    right_on = ['FIPS'],
                    how = 'inner')

In [0]:
cases_pop['Cases per 1000 people'] = round(1000*cases_pop['total_cases']/cases_pop['Total Population'],1)
cases_pop['Hospitalizations per 1000 people'] = round(1000*cases_pop['hospitalizations']/cases_pop['Total Population'],1)
cases_pop['Deaths per 1000 people'] = round(1000*cases_pop['deaths']/cases_pop['Total Population'],1)

### Choropleth map and table

In [0]:
r = requests.get('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
counties = json.loads(r.text)

In [0]:
fig = px.choropleth(cases_pop, geojson=counties, locations='fips', 
                    color='Cases per 1000 people',
                    color_continuous_scale="Viridis",
                    scope="usa",
                    labels={'total_cases':'Cases per 1000 people'},
                    hover_name = 'locality'
                   )
fig.update_geos(fitbounds="locations")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [0]:
cases_table = cases_pop[['locality', 'Cases per 1000 people',
                         'Hospitalizations per 1000 people',
                         'Deaths per 1000 people']]
dash.dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True} for i in cases_table.columns
        ],
        data=cases_table,
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10)

### Create the trend dataset

In [0]:
cases_trend = cases[['report_date', 'fips', 'locality', 'total_cases', 'hospitalizations', 'deaths']]
cases_trend = pd.melt(cases_trend, id_vars = ['report_date', 'fips', 'locality'],
                     value_vars = ['total_cases', 'hospitalizations', 'deaths'])
cases_trend['date14'] = cases_trend['report_date'] + timedelta(14)
cases_trend['date28'] = cases_trend['report_date'] + timedelta(28)

cases_trend = pd.merge(cases_trend, cases_trend,
                      right_on = ['report_date', 'fips', 'locality', 'variable'],
                      left_on = ['date14', 'fips', 'locality', 'variable'])

cases_trend = cases_trend.drop(['report_date_x','date14_x','date28_x'], axis=1)
cases_trend = cases_trend.rename({'report_date_y':'report_date',
                                 'date14_y':'date14',
                                 'date28_y':'date28',
                                 'value_y':'value',
                                 'value_x':'value14'}, axis=1)

cases_trend = pd.merge(cases_trend, cases_trend,
                      right_on = ['report_date', 'fips', 'locality', 'variable'],
                      left_on = ['date28', 'fips', 'locality', 'variable'])

cases_trend = cases_trend.drop(['report_date_x','date14_y','date28_y', 'value14_x', 'date14_x', 'date28_x'], axis=1)
cases_trend = cases_trend.rename({'report_date_y':'report_date',
                                 'value_y':'value',
                                 'value14_y':'value14',
                                 'value_x':'value28'}, axis=1)
#cases_trend = cases_trend.loc[cases_trend['report_date'] == max(cases_trend['report_date'])]
cases_trend['Most recent 14 days'] = cases_trend['value'] - cases_trend['value14']
cases_trend['Previous 14 days'] = cases_trend['value14'] - cases_trend['value28']
cases_trend['Trend'] = round(100*(cases_trend['Most recent 14 days'] - cases_trend['Previous 14 days']) / cases_trend['Previous 14 days'], 1)
cases_trend['Trend'] = cases_trend['Trend'].astype('str') + "%"
cases_trend

### Line plot and table for one locality

In [0]:
loc = 'Alleghany'
today = max(cases_trend['report_date']).strftime('%B %d, %Y')
title = f"COVID Statistics for {loc} through {today}"
cases_loc = cases_trend.query(f"locality == '{loc}'")
fig = px.line(cases_loc, x='report_date', y='value', color='variable', facet_row='variable', 
                 hover_data=['Most recent 14 days', 'Previous 14 days', 'Trend'],
                 labels={'report_date':'Date', 
                      'value':'Total'},
                 title = title,
                 width=1000, height=800)
fig.update(layout=dict(title=dict(x=0.5)))
fig.update_layout(showlegend=False)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("variable=", "")))
fig.update_yaxes(matches=None)
fig.show()

In [0]:
tab = cases_loc.loc[cases_loc['report_date'] == max(cases_loc['report_date'])][['variable', 
                                                                          'Most recent 14 days',
                                                                         'Previous 14 days',
                                                                         'Trend']]

In [0]:
dash.dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True} for i in tab.columns
        ],
        data=tab,
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10)